# Project Notebook




The task is to produce a neural process which is tailored to solve a physical system.  In this case, the physical system to be solved is the heat diffusion across a two-dimensional grid.  There are three main changes I wish to make to the standard neural process.  These are listed below, along with a brief description of their physical motivation.

#### 1. Convolutional Encoder and Decoder block 
Spatial equivariance is built into the system, since the heat diffusion function does not have a spatially dependent forcing term.  All terms are based on derivatives, which means the solution obeys spatial equivariance.  I hope to replicate this feature in the architecture, allowing for an easier learning process.
#### 2. Learning using Privileged Information
Learning using privileged information (LUPI) allows the model, while training, to have access to `privileged information' (PI).  PI could be anything important the model should be aware of.  For example, if the total energy in the grid is conserved, imput this as PI.  When it comes to test time, the model does not have access to any PI.  The hope is that the model can infer what the PI should be (e.g. by figuring out what the total energy is and conserving it), making for a faster learning process.  There is a wide variety of PI to be used and experimented with.  A few more examples are: initial conditions, final conditions, decay constant, entropy measure.

The way the PI is implemented is through an encoder which runs separately to the usual encoder.  The PI embedding is then aggregated with the embedding of the data using a residual neural network; we do not want the PI to dominate the learning, but merely to add a `first order correction' to the learning process.  The good thing about this structure is that the PI can be added to any encoder block (e.g. flat encoder or convolutional encoder)

#### 3. Time-Order Respecting Aggregator
In a neural process, the embeddings of the data at different times, $t_i$, are aggregated with a permutation invariant aggregator.  This is not usually useful for physical processes, as there tends to be a time-respecting process such as increase in entropy (in this case, diffusion of heat).  Therefore, it might be useful to reflect this in the aggregator.  The aggregator is such that there is no information leak from future to past - it imposes causality.

The aggregator could also be designed to reflect temporal equivariance, for the same reasons as the spatial equivariance: the diffusion equation involves no terms which depend explicitly on time, but rather only the derivative with resepct to time.  If the model is temporally equivariant, it may perform much better on extrapolation tasks (as it is not completely `blind' when it heads out of the sample, having learnt temporal behaviour)

One possible choice for this is to allow the model to `learn' these features by implementing a multi-head attention network.  However, with this option there is far less control over which physical concepts I am imposing.


I hope these three changes will improve on the most basic vanilla neural process architecture to learn the heat diffusion behaviour over a two-dimensional grid, as they are all grounded in physical concepts. 

In [2]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from scipy.integrate import odeint


# grab the neural process functions from Emilien Dupont's library called neural-processes
import sys

sys.path.insert(0, "./neural-processes")

# neural_process.py and training.py exist in neural-processes folder
import neural_process
from neural_process import NeuralProcessConv, NeuralProcess, NeuralProcessImg
from training import NeuralProcessTrainer
from utils import context_target_split
from heat_diffusion_dataset import Initial_Conditions, Diffusion_Data, RestoredData


# Diffusion Data

In [3]:
dimensions = {
    't_dim' : 1,
    'y_dim' : 2500,
    'r_dim' : 500,
    'z_dim' : 500,
    'h_dim' : 500,
    'z_dim' : 500,
    'pi_dim' : 4,
    'max_iter_time' : 100,
    'grid_size' : 50,
    'num_samples' : 40,  # why do I need this one...
    'num_context' : 40,
    'num_target' : 40,
    'batch_size' : 2,
    'num_channels' : 512,
    'tcn_kernel_size' : 5,
}

assert dimensions['grid_size']**2 == dimensions['y_dim']
assert dimensions['tcn_kernel_size'] % 2 == 1

# parameters
grid_size = dimensions['grid_size']
num_samples = dimensions['num_samples']
square_range = (5, 10)
temp_range = (0.7, 1.0)
diffusion_coef = 0.25

In [2]:
''' 
RUN TO RE-GENERATE DATA
'''

initial_conditions = Initial_Conditions(
    max_iter_time=dimensions['max_iter_time'], grid_size=grid_size
)

process_dataset = Diffusion_Data(
    num_samples=num_samples,
    max_iter_time=dimensions['max_iter_time'],
    grid_size=grid_size,
    initial_conditions=initial_conditions,
    square_range=square_range,
    temp_range=temp_range,
    diffusion_coef=diffusion_coef,
    pi_dim=dimensions['pi_dim']
)

# plot target data
# process_dataset.visualise_solution(max_iter_time=dimensions['max_iter_time'], show_until=30)

time_array = process_dataset.time_array

# process_dataset.save_tensors()

In [5]:
''' 
RUN TO READ DATA
'''

dataset = RestoredData(
    num_samples=dimensions['num_samples'],
    max_iter_time=dimensions['max_iter_time'],
    grid_size=dimensions['grid_size'],
    temp_range=temp_range
)

pure_noise = dataset.generate_noise()

time_array = dataset.time_array

## VANILLA NEURAL PROCESS TRAINING

In [6]:
neuralprocess = NeuralProcess(dimensions)

data_loader = DataLoader(dataset, batch_size=dimensions['batch_size'], shuffle=True)
optimizer = torch.optim.Adam(neuralprocess.parameters(), lr=3e-4)
np_trainer = NeuralProcessTrainer(
    "cuda",
    neuralprocess,
    optimizer,
    num_context_range=(dimensions['num_context'], dimensions['num_context']),
    num_extra_target_range=(dimensions['num_target'], dimensions['num_target']),
    print_freq=200,
)

neuralprocess.training = True
np_trainer.train(data_loader, 1000)

Epoch: 0, Avg_loss: 78513.87935791016
Epoch: 1, Avg_loss: 16173.517932128907


KeyboardInterrupt: 

## CONVOLUTIONAL NEURAL PROCESS TRAINING

#### Hyperparameters
- h_dim
- r_dim
- z_dim
- batch_size
- num_context
- num_target
- lr

#### Hidden Hyperparameters
- Number of linear layers after convolution in Net
- Number of linear layers in TransposeConvNet
- Number of channels in Net and TransposeConvNet
- Number of layers in Net and TransposeConvNet
- Kernels, Dilations, Paddings

#### Current Convolution Summary 

Net: 2 layers with 32 'hidden' channels, followed by 3 Linear layers with h_dim hidden.

TNet: 1 Linear layer, followed by 2 transpose convolutional layers with 32 'hidden' channels.


#### Purpose of the Convolution
- Create spatially equivariant features in the grids
- Encourage principles of locality when learning
- Temporal equivariance in solution (periodicity encouraged?) - this is not happening currently as the temporal aspect only enters after the convolution

TODO: before moving on 
- Debug the conv neural process (write some tests?)
- Implement tests on standard NP 
- Compare performance  

In [7]:
""" 
TODO 
- put time into channels not flat layer DONE - also kept in flat layer
- play with hyperparams until good outcome 
"""

conv_neuralprocess = NeuralProcessConv(
    dimensions['t_dim'], dimensions['y_dim'], dimensions['r_dim'], dimensions['z_dim'], dimensions['h_dim'], dimensions['grid_size']
)

data_loader = DataLoader(dataset, batch_size=dimensions['batch_size'], shuffle=True)
optimizer = torch.optim.Adam(conv_neuralprocess.parameters(), lr=3e-4)
conv_np_trainer = NeuralProcessTrainer(
    "cuda",
    conv_neuralprocess,
    optimizer,
    num_context_range=(dimensions['num_context'], dimensions['num_context']),
    num_extra_target_range=(dimensions['num_target'], dimensions['num_target']),
    print_freq=200,
    grid_size=grid_size,
)

conv_neuralprocess.training = True
conv_np_trainer.train(data_loader, 120)

TypeError: __init__() takes 2 positional arguments but 7 were given

## L.U.P.I. FOR FLAT NP

In [8]:
neuralprocess_pi = NeuralProcess(dimensions)

data_loader = DataLoader(dataset, batch_size=dimensions['batch_size'], shuffle=True)
optimizer = torch.optim.Adam(neuralprocess_pi.parameters(), lr=3e-4)
np_trainer_pi = NeuralProcessTrainer(
    "cuda",
    neuralprocess_pi,
    optimizer,
    num_context_range=(dimensions['num_context'], dimensions['num_context']),
    num_extra_target_range=(dimensions['num_target'], dimensions['num_target']),
    print_freq=200,
)

neuralprocess_pi.training = True
np_trainer_pi.train(data_loader, 1000)

Epoch: 0, Avg_loss: 85659.2748046875
Epoch: 1, Avg_loss: -10707.619189453126


KeyboardInterrupt: 

# TESTING

In [11]:
from utils import context_target_split

# Select which models to test
test_convolutional_neural_process = False
test_lupi_neural_process = False
test_flat_neural_process = True

# Extract a batch from data_loader
for batch in data_loader:
    break

# Use batch to create random set of context points
t, y, pi = batch
t_context, y_context, _, _, _, _ = context_target_split(
    t[0:1], y[0:1], pi[0:1], dimensions['num_context'], dimensions['num_target']
)

# Create a set of target points corresponding to in-sample time range
t_target = torch.linspace(0, 1, dimensions['max_iter_time'])
t_target = t_target.unsqueeze(1).unsqueeze(0)

# Extract mean of distribution
t_out = t_target.numpy()[0]

# target output
y_target = y[0:1][0, :, :]
y_target = y_target.reshape((dimensions['max_iter_time'], dimensions['grid_size'], dimensions['grid_size']))

if test_lupi_neural_process:
    # Switch to test mode
    neuralprocess_pi.training = False

    # Extract models' distributions over t_target and grid
    p_y_pred = neuralprocess_pi(t_context, y_context, t_target)

    # flat NP
    y_out = p_y_pred.loc.detach().numpy()[0]
    var_out = p_y_pred.scale.detach().numpy()[0]

    # flat NP reshape
    y_out = y_out.reshape((dimensions['max_iter_time'], dimensions['grid_size'], dimensions['grid_size']))
    var_out = var_out.reshape((dimensions['max_iter_time'], dimensions['grid_size'], dimensions['grid_size']))

    # flat NP result summary
    result = [
        (y_target[id, :, :], y_out[id, :, :], var_out[id, :, :])
        for id in range(y_target.shape[0])
    ]

    torch.save(result, "./results/result.pt")

if test_flat_neural_process:
    # Switch to test mode
    neuralprocess.training = False

    # Extract models' distributions over t_target and grid
    p_y_pred = neuralprocess(t_context, y_context, t_target)

    # flat NP
    y_out = p_y_pred.loc.detach().numpy()[0]
    var_out = p_y_pred.scale.detach().numpy()[0]

    # flat NP reshape
    y_out = y_out.reshape((dimensions['max_iter_time'], dimensions['grid_size'], dimensions['grid_size']))
    var_out = var_out.reshape((dimensions['max_iter_time'], dimensions['grid_size'], dimensions['grid_size']))

    # flat NP result summary
    result = [
        (y_target[id, :, :], y_out[id, :, :], var_out[id, :, :])
        for id in range(y_target.shape[0])
    ]

    torch.save(result, "./results/np_result.pt")


if test_convolutional_neural_process:
    conv_neuralprocess.training = False

    conv_p_y_pred = conv_neuralprocess(t_context, y_context, t_target)

    # convolutional NP
    conv_y_out = conv_p_y_pred.loc.detach().numpy()[0]
    conv_var_out = conv_p_y_pred.scale.detach().numpy()[0]
    # convolutional NP reshape
    conv_y_out = conv_y_out.reshape((dimensions['max_iter_time'], dimensions['grid_size'], dimensions['grid_size']))
    conv_var_out = conv_var_out.reshape((dimensions['max_iter_time'], dimensions['grid_size'], dimensions['grid_size']))
    # convolutional NP result summary
    conv_result = [
        (y_target[id, :, :], conv_y_out[id, :, :], conv_var_out[id, :, :])
        for id in range(y_target.shape[0])
    ]
    torch.save(conv_result, "./results/conv_result.pt")

# Understanding Model Learning Processes

#### Input noise into the model and see what its characteristics are 
#### Plot the losses as as function of epochs


In [6]:
# Switch to test mode
neuralprocess.training = False

# Extract models' distributions over t_target and grid
p_y_pred = neuralprocess(t_context, y_context, t_target)

# flat NP
y_out = p_y_pred.loc.detach().numpy()[0]
var_out = p_y_pred.scale.detach().numpy()[0]

# flat NP reshape
y_out = y_out.reshape((max_iter_time, grid_size, grid_size))
var_out = var_out.reshape((max_iter_time, grid_size, grid_size))

# flat NP result summary
result = [
    (y_target[id, :, :], y_out[id, :, :], var_out[id, :, :])
    for id in range(y_target.shape[0])
]

'''
Loss Plots
'''

epoch_loss = torch.Tensor(np_trainer.epoch_loss_history)
torch.save(epoch_loss, "./results/epoch_loss_flat.pt")

# epoch_loss_pi = torch.Tensor(np_trainer_pi.epoch_loss_history)
# torch.save(epoch_loss, "./results/epoch_loss_pi.pt")

# epoch_loss_conv = torch.Tensor(conv_np_trainer.epoch_loss_history)
# torch.save(epoch_loss, "./results/epoch_loss_conv.pt")


# Fix this bug for GIF

In [ ]:
y_out = y_out.reshape((max_iter_time, grid_size, grid_size))
var_out = var_out.reshape((max_iter_time, grid_size, grid_size))
y_target = y_target.reshape((max_iter_time, grid_size, grid_size))

result = [
    (y_target[id, :, :], y_out[id, :, :], var_out[id, :, :])
    for id in range(y_target.shape[0])
]

# from heat_diffusion_dataset import Initial_Conditions, Diffusion_Data
# import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation


def plotheatmap(result_k, k, axes):
    # Clear the current plot figure
    plt.clf()
    ax1, ax2, ax3 = axes

    # plt.title(f"Temperature at t = {k * delta_t:.3f} unit time")
    ax1.set_title(f"Target at t = {k * 4 * np.pi / max_iter_time:.3f} unit time")
    ax1.set_xlabel("x")
    ax1.set_ylabel("y")

    ax2.set_title(
        f"Predicted Mean at t = {k * 4 * np.pi / max_iter_time:.3f} unit time"
    )
    ax2.set_xlabel("x")
    ax2.set_ylabel("y")

    ax3.set_title(f"Predicted Var at t = {k * 4 * np.pi / max_iter_time:.3f} unit time")
    ax3.set_xlabel("x")
    ax3.set_ylabel("y")

    # This is to plot u_k (u at time-step k)
    ax1.pcolormesh(result_k[0], cmap=plt.cm.jet, vmin=-1.1, vmax=1.1)
    ax2.pcolormesh(result_k[1], cmap=plt.cm.jet, vmin=-1.1, vmax=1.1)
    ax3.pcolormesh(result_k[2], cmap=plt.cm.jet, vmin=-1.1, vmax=1.1)

    return fig, axes


def animate(k):
    plotheatmap(result[k], k)

    # if not plot_var and not target:
    #     plotheatmap(y_out[k], k, delta_t)
    # elif plot_var and not target:
    #     plotheatmap(var_out[k], k, delta_t)
    # else:
    #     plotheatmap(y_target[k], k, delta_t)


# plot_var = False
# target = False
fig, axes = plt.subplots(ncols=3, figsize=(15, 4))

anim = animation.FuncAnimation(
    fig, animate, interval=1, frames=max_iter_time, repeat=False, fargs=(axes,)
)
anim.save("harmonics_solution.gif")

# plot_var=True
# anim = animation.FuncAnimation(plt.figure(), animate, interval=1, frames=max_iter_time, repeat=False, fargs=None)
# anim.save("harmonics_variance.gif")

# target=True
# anim = animation.FuncAnimation(plt.figure(), animate, interval=1, frames=max_iter_time, repeat=False, fargs=None)
# anim.save("harmonics_target.gif")